In [1]:
import numpy as np
import os
import tensorflow as tf
from skimage.transform import resize
import matplotlib.pyplot as plt

2025-06-06 19:13:21.354522: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-06 19:13:21.391855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-06 19:13:21.391878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-06 19:13:21.392687: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-06 19:13:21.398451: I tensorflow/core/platform/cpu_feature_guar

In [3]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# We need the fine-tuned model for feature extraction

In [5]:
# load the finetuned gmp model
gmp_model = tf.keras.models.load_model('../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5')

2025-06-06 19:13:49.221124: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [6]:
gmp_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [7]:
# define the feature extractor

# feature extractor input
feat_ext_input = gmp_model.input

# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = gmp_model.layers[-4].output

feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [8]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [9]:
# From the previous work, we have the following functions that we have written and can use directly for the work here

In [10]:
# Write a function for extracting onlt the horizontal images out of all the files in the location

def get_horizontal_images(file_path):
    # get all contents at the file path
    all_files = os.listdir(file_path)
    # get only the image files as we do not need the xml files for the seq2seq model
    only_images = [file for file in all_files if file.split(".")[-1] == 'jpeg']
    # from all the images only choose the horizontal ones
    horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
    # sort the list
    horizontal_images.sort()
    # return the image list
    return horizontal_images

In [11]:
# We need to preprocess all the data for the rest of the blocks, for the task of prediction and performance evaluation.

# The problem we currently have is the written code takes up a lot of time. -It takes too much time to do the feature extraction. Currently we are using the model

In [12]:
# The following alternate functions are written to improve the execution time of the prediction from the model to extract features

In [13]:
def create_sub_windows(folder_path, file, stride = 30, kernel_size = 30):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch all subwindows here
    all_subwindows = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # append these to the list
            all_subwindows.append(sub_window)
            
    return all_subwindows  

In [14]:
def extract_features(file_name, file_path):
    # get the subwindows
    subwindows = create_sub_windows(file_path, file_name, 30, 30)
    # stack the subwindows
    stacked_subwindows = np.stack(subwindows, axis = 0)
    # print the shape of this
    print(stacked_subwindows.shape)
    # extract features
    extracted_featrues = feature_extractor_model.predict(stacked_subwindows)
    
    return extracted_featrues

In [15]:
# The images we use in the task are in the following list

In [16]:
horizontal_image_list = ['2020_08_03', '2020_08_04', '2020_08_06', '2020_08_07', '2020_08_11', '2020_08_12', '2020_08_14', '2020_08_15', '2020_08_17', '2020_08_18', '2020_08_19', '2020_08_21', '2020_08_25', '2020_08_26', '2020_08_27', '2020_08_28', '2020_08_31', '2020_09_02', '2020_09_07', '2020_09_16']

In [17]:
# Okay, let's try and solve this problem with the first block in the test dataset

Block 0103

In [18]:
blk_0103_data_path = '../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml'

In [19]:
# we can still use the function above to get the horizontal images
blk_0103_horizontal_images = get_horizontal_images(blk_0103_data_path)

In [20]:
# separate the inputs and the targets
blk_0103_input_time_point_images = blk_0103_horizontal_images[:13]
blk_0103_input_time_point_images.sort()
blk_0103_output_time_point_images = blk_0103_horizontal_images[-7:]
blk_0103_output_time_point_images.sort()

In [21]:
# blk_0103_input_time_point_images, blk_0103_output_time_point_images

In [22]:
# Okay, now we need to modify the next function a little to see if the timing for the computations can be improved

In [23]:
# What if we first stack the subwindows in a function, and then do model.predict may be in a separate function

In [24]:
def create_sub_windows(folder_path, file, stride = 30, kernel_size = 30):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch all subwindows here
    all_subwindows = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # append these to the list
            all_subwindows.append(sub_window)
            
    return all_subwindows  

In [25]:
# Try this for a single image

In [26]:
test_image_0 = blk_0103_input_time_point_images[0]

In [27]:
test_image_0_sub_windows = create_sub_windows(blk_0103_data_path, test_image_0, 30, 30)

In [28]:
# need to stack the list
test_image_0_sub_windows_stack = np.stack(test_image_0_sub_windows, axis = 0)

In [29]:
test_image_0_sub_windows_stack.shape

(910, 30, 30, 3)

In [30]:
%%time
# try model.predict with this
test_image_0_extracted_features = feature_extractor_model.predict(test_image_0_sub_windows_stack)

2025-06-06 19:14:19.271290: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


29/29 [==============================] - 2s 5ms/step
CPU times: user 605 ms, sys: 234 ms, total: 838 ms
Wall time: 2.16 s


In [31]:
test_image_0_extracted_features.shape

(910, 32)

In [32]:
# Okay, so this happense pretty fast (in 2 seconds as opposed to 42 secods - so we do have a huge improvemet in execution time)

In [33]:
# Try doing this for all images

In [34]:
# input features for block 0103
blk_0103_input_time_point_images

['Block0103_2020_08_03.jpeg',
 'Block0103_2020_08_04.jpeg',
 'Block0103_2020_08_06.jpeg',
 'Block0103_2020_08_07.jpeg',
 'Block0103_2020_08_11.jpeg',
 'Block0103_2020_08_12.jpeg',
 'Block0103_2020_08_14.jpeg',
 'Block0103_2020_08_15.jpeg',
 'Block0103_2020_08_17.jpeg',
 'Block0103_2020_08_18.jpeg',
 'Block0103_2020_08_19.jpeg',
 'Block0103_2020_08_21.jpeg',
 'Block0103_2020_08_25.jpeg']

In [35]:
%%time
block_0103_extracted_features = []
for file in blk_0103_input_time_point_images:
    # get the subwindows
    subwindows = create_sub_windows(blk_0103_data_path, file, 30, 30)
    # stack the subwindows
    stacked_subwindows = np.stack(subwindows, axis = 0)
    # print the shape of this
    print(stacked_subwindows.shape)
    # extract features
    extracted_featrues = feature_extractor_model.predict(stacked_subwindows)
    # append the extracted features
    block_0103_extracted_features.append(extracted_featrues)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.63 s, sys: 206 ms, total: 4.84 s
Wall time: 5.2 s


In [36]:
block_0103_extracted_features[0].shape

(910, 32)

In [37]:
# now stack all these together?
blk_0103_stacked_input_features = np.stack(block_0103_extracted_features, axis = 1)

In [38]:
blk_0103_stacked_input_features.shape

(910, 13, 32)

In [39]:
# Okay, this seems to work really fast - better!
# now what if we do a function to get the predictions (extracted features too) - this would be easy for the rest of the blocks

In [40]:
def extract_features(file_name, file_path):
    # get the subwindows
    subwindows = create_sub_windows(file_path, file_name, 30, 30)
    # stack the subwindows
    stacked_subwindows = np.stack(subwindows, axis = 0)
    # print the shape of this
    print(stacked_subwindows.shape)
    # extract features
    extracted_featrues = feature_extractor_model.predict(stacked_subwindows)
    
    return extracted_featrues

In [41]:
blk_0103_data_path

'../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml'

In [42]:
%%time
# see if this works
blk_0103_alt_all_features = []
for file in blk_0103_input_time_point_images:
    extracted_features = extract_features(file, blk_0103_data_path)
    blk_0103_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 1ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 1ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.46 s, sys: 288 ms, total: 4.75 s
Wall time: 4.6 s


In [43]:
alt_features_stack_0103 = np.stack(blk_0103_alt_all_features, axis = 1)

In [44]:
alt_features_stack_0103.shape

(910, 13, 32)

In [45]:
# check if these are the same
np.mean(alt_features_stack_0103 == blk_0103_stacked_input_features)

1.0

In [46]:
# also let's get the means
np.mean(alt_features_stack_0103), np.mean(blk_0103_stacked_input_features)

(0.34708995, 0.34708995)

In [47]:
# Let's do a sanity check here

In [48]:
# What sort of sanity check should we do? Maybe try to track down a few random arrays to ensure the stacking has happened correctly?

In [49]:
len(blk_0103_alt_all_features)

13

In [50]:
blk_0103_alt_all_features[0].shape

(910, 32)

In [51]:
catch_all = []
for i in range(len(blk_0103_alt_all_features)):
    mean_vec = np.mean(blk_0103_alt_all_features[i] == alt_features_stack_0103[:,i, :])
    catch_all.append(mean_vec)
    print(mean_vec)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [52]:
np.mean(catch_all)

1.0

In [53]:
# I think this is enough sanity checks. We also checked for output np arrays from both preprocessing methods in notebook 4. Let's just make sure the preprocessing is done correctly for the remainig blocks

In [54]:
# Let's use only the functions we developed above to get the extracted features for block 0103

Block 0103

In [55]:
blk_0103_data_path = '../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml'

In [56]:
# we can still use the function above to get the horizontal images
blk_0103_horizontal_images = get_horizontal_images(blk_0103_data_path)

In [57]:
# separate the inputs and the targets
blk_0103_input_time_point_images = blk_0103_horizontal_images[:13]
blk_0103_input_time_point_images.sort()
blk_0103_output_time_point_images = blk_0103_horizontal_images[-7:]
blk_0103_output_time_point_images.sort()

In [58]:
%%time
# get the extracted input features

blk_0103_alt_all_features = []
for file in blk_0103_input_time_point_images:
    extracted_features = extract_features(file, blk_0103_data_path)
    blk_0103_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 1ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.48 s, sys: 227 ms, total: 4.71 s
Wall time: 4.54 s


In [59]:
# stack the extracted features
blk_0103_stacked_extracted_input_features = np.stack(blk_0103_alt_all_features, axis = 1)

In [60]:
blk_0103_stacked_extracted_input_features.shape

(910, 13, 32)

In [61]:
# check if mean matches
np.mean(blk_0103_stacked_extracted_input_features)

0.34708995

In [60]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0103_extracted_input_features.npy", blk_0103_stacked_extracted_input_features)

In [62]:
# extract the targets

In [63]:
%%time
# get the extracted targets

blk_0103_targets = []
for file in blk_0103_output_time_point_images:
    extracted_features = extract_features(file, blk_0103_data_path)
    blk_0103_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.45 s, sys: 127 ms, total: 2.58 s
Wall time: 2.78 s


In [64]:
# stack the extracted features
blk_0103_stacked_extracted_targets = np.stack(blk_0103_targets, axis = 1)

In [65]:
blk_0103_stacked_extracted_targets.shape

(910, 7, 32)

In [65]:
# Store this stack
np.save("seq_2_seq_test_data/block_0103_extracted_target_features.npy", blk_0103_stacked_extracted_targets)

Block 0104

In [66]:
blk_0104_data_path = '../../Spring_2024/S_lab_TasselNet/Block_4_TN/Block_4_images_and_xml'

In [67]:
# we can still use the function above to get the horizontal images
blk_0104_horizontal_images = get_horizontal_images(blk_0104_data_path)

In [68]:
# separate the inputs and the targets
blk_0104_input_time_point_images = blk_0104_horizontal_images[:13]
blk_0104_input_time_point_images.sort()
blk_0104_output_time_point_images = blk_0104_horizontal_images[-7:]
blk_0104_output_time_point_images.sort()

In [69]:
# blk_0104_input_time_point_images, blk_0104_output_time_point_images

In [70]:
%%time
# get the extracted input features

blk_0104_alt_all_features = []
for file in blk_0104_input_time_point_images:
    extracted_features = extract_features(file, blk_0104_data_path)
    blk_0104_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 1ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.55 s, sys: 258 ms, total: 4.8 s
Wall time: 4.98 s


In [71]:
# stack the extracted features
blk_0104_stacked_extracted_input_features = np.stack(blk_0104_alt_all_features, axis = 1)

In [72]:
blk_0104_stacked_extracted_input_features.shape

(910, 13, 32)

In [73]:
# check if mean matches
np.mean(blk_0104_stacked_extracted_input_features)

0.38871154

In [74]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0104_extracted_input_features.npy", blk_0104_stacked_extracted_input_features)

In [75]:
# extract the targets

In [76]:
%%time
# get the extracted targets

blk_0104_targets = []
for file in blk_0104_output_time_point_images:
    extracted_features = extract_features(file, blk_0104_data_path)
    blk_0104_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.48 s, sys: 125 ms, total: 2.6 s
Wall time: 2.49 s


In [77]:
# stack the extracted features
blk_0104_stacked_extracted_targets = np.stack(blk_0104_targets, axis = 1)

In [78]:
blk_0104_stacked_extracted_targets.shape

(910, 7, 32)

In [79]:
# Store this stack
np.save("seq_2_seq_test_data/block_0104_extracted_target_features.npy", blk_0104_stacked_extracted_targets)

Block 0105

In [80]:
blk_0105_data_path = '../../Spring_2024/S_lab_TasselNet/Block_5_TN/Block_5_images_and_xml'

In [81]:
# we can still use the function above to get the horizontal images
blk_0105_horizontal_images = get_horizontal_images(blk_0105_data_path)

In [82]:
# separate the inputs and the targets
blk_0105_input_time_point_images = blk_0105_horizontal_images[:13]
blk_0105_input_time_point_images.sort()
blk_0105_output_time_point_images = blk_0105_horizontal_images[-7:]
blk_0105_output_time_point_images.sort()

In [83]:
# blk_0105_input_time_point_images, blk_0105_output_time_point_images

In [84]:
%%time
# get the extracted input features

blk_0105_alt_all_features = []
for file in blk_0105_input_time_point_images:
    extracted_features = extract_features(file, blk_0105_data_path)
    blk_0105_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.59 s, sys: 253 ms, total: 4.84 s
Wall time: 4.61 s


In [85]:
# stack the extracted features
blk_0105_stacked_extracted_input_features = np.stack(blk_0105_alt_all_features, axis = 1)

In [86]:
blk_0105_stacked_extracted_input_features.shape

(910, 13, 32)

In [87]:
# check if mean matches
np.mean(blk_0105_stacked_extracted_input_features)

0.36304483

In [88]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0105_extracted_input_features.npy", blk_0105_stacked_extracted_input_features)

In [89]:
# extract the targets

In [90]:
%%time
# get the extracted targets

blk_0105_targets = []
for file in blk_0105_output_time_point_images:
    extracted_features = extract_features(file, blk_0105_data_path)
    blk_0105_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.41 s, sys: 160 ms, total: 2.57 s
Wall time: 2.45 s


In [91]:
# stack the extracted features
blk_0105_stacked_extracted_targets = np.stack(blk_0105_targets, axis = 1)

In [92]:
blk_0105_stacked_extracted_targets.shape

(910, 7, 32)

In [93]:
# Store this stack
np.save("seq_2_seq_test_data/block_0105_extracted_target_features.npy", blk_0105_stacked_extracted_targets)

Block 0106

In [94]:
blk_0106_data_path = '../../Spring_2024/S_lab_TasselNet/Block_6_TN/Block_6_images_and_xml'

In [95]:
# we can still use the function above to get the horizontal images
blk_0106_horizontal_images = get_horizontal_images(blk_0106_data_path)

In [96]:
# separate the inputs and the targets
blk_0106_input_time_point_images = blk_0106_horizontal_images[:13]
blk_0106_input_time_point_images.sort()
blk_0106_output_time_point_images = blk_0106_horizontal_images[-7:]
blk_0106_output_time_point_images.sort()

In [97]:
# blk_0106_input_time_point_images, blk_0106_output_time_point_images

In [98]:
%%time
# get the extracted input features

blk_0106_alt_all_features = []
for file in blk_0106_input_time_point_images:
    extracted_features = extract_features(file, blk_0106_data_path)
    blk_0106_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.55 s, sys: 251 ms, total: 4.8 s
Wall time: 4.58 s


In [99]:
# stack the extracted features
blk_0106_stacked_extracted_input_features = np.stack(blk_0106_alt_all_features, axis = 1)

In [100]:
blk_0106_stacked_extracted_input_features.shape

(910, 13, 32)

In [101]:
# check if mean matches
np.mean(blk_0106_stacked_extracted_input_features)

0.3736453

In [102]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0106_extracted_input_features.npy", blk_0106_stacked_extracted_input_features)

In [103]:
# extract the targets

In [104]:
%%time
# get the extracted targets

blk_0106_targets = []
for file in blk_0106_output_time_point_images:
    extracted_features = extract_features(file, blk_0106_data_path)
    blk_0106_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.37 s, sys: 128 ms, total: 2.49 s
Wall time: 2.39 s


In [105]:
# stack the extracted features
blk_0106_stacked_extracted_targets = np.stack(blk_0106_targets, axis = 1)

In [106]:
blk_0106_stacked_extracted_targets.shape

(910, 7, 32)

In [107]:
# Store this stack
np.save("seq_2_seq_test_data/block_0106_extracted_target_features.npy", blk_0106_stacked_extracted_targets)

Block 0201

In [108]:
blk_0201_data_path = '../../Spring_2024/S_lab_TasselNet/Block_7_TN/Block_7_images_and_xml'

In [109]:
# we can still use the function above to get the horizontal images
blk_0201_horizontal_images = get_horizontal_images(blk_0201_data_path)

In [110]:
# separate the inputs and the targets
blk_0201_input_time_point_images = blk_0201_horizontal_images[:13]
blk_0201_input_time_point_images.sort()
blk_0201_output_time_point_images = blk_0201_horizontal_images[-7:]
blk_0201_output_time_point_images.sort()

In [111]:
# blk_0201_input_time_point_images, blk_0201_output_time_point_images

In [112]:
%%time
# get the extracted input features

blk_0201_alt_all_features = []
for file in blk_0201_input_time_point_images:
    extracted_features = extract_features(file, blk_0201_data_path)
    blk_0201_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.48 s, sys: 266 ms, total: 4.75 s
Wall time: 4.54 s


In [113]:
# stack the extracted features
blk_0201_stacked_extracted_input_features = np.stack(blk_0201_alt_all_features, axis = 1)

In [114]:
blk_0201_stacked_extracted_input_features.shape

(910, 13, 32)

In [115]:
# check if mean matches
np.mean(blk_0201_stacked_extracted_input_features)

0.35994256

In [116]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0201_extracted_input_features.npy", blk_0201_stacked_extracted_input_features)

In [117]:
# extract the targets

In [118]:
%%time
# get the extracted targets

blk_0201_targets = []
for file in blk_0201_output_time_point_images:
    extracted_features = extract_features(file, blk_0201_data_path)
    blk_0201_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.47 s, sys: 148 ms, total: 2.62 s
Wall time: 2.5 s


In [119]:
# stack the extracted features
blk_0201_stacked_extracted_targets = np.stack(blk_0201_targets, axis = 1)

In [120]:
blk_0201_stacked_extracted_targets.shape

(910, 7, 32)

In [121]:
# Store this stack
np.save("seq_2_seq_test_data/block_0201_extracted_target_features.npy", blk_0201_stacked_extracted_targets)

Block 0202

In [122]:
blk_0202_data_path = '../../Spring_2024/S_lab_TasselNet/Block_8_TN/Block_8_images_and_xml'

In [123]:
# we can still use the function above to get the horizontal images
blk_0202_horizontal_images = get_horizontal_images(blk_0202_data_path)

In [124]:
# separate the inputs and the targets
blk_0202_input_time_point_images = blk_0202_horizontal_images[:13]
blk_0202_input_time_point_images.sort()
blk_0202_output_time_point_images = blk_0202_horizontal_images[-7:]
blk_0202_output_time_point_images.sort()

In [125]:
# blk_0202_input_time_point_images, blk_0202_output_time_point_images

In [126]:
%%time
# get the extracted input features

blk_0202_alt_all_features = []
for file in blk_0202_input_time_point_images:
    extracted_features = extract_features(file, blk_0202_data_path)
    blk_0202_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.5 s, sys: 137 ms, total: 4.63 s
Wall time: 4.42 s


In [127]:
# stack the extracted features
blk_0202_stacked_extracted_input_features = np.stack(blk_0202_alt_all_features, axis = 1)

In [128]:
blk_0202_stacked_extracted_input_features.shape

(910, 13, 32)

In [129]:
# check if mean matches
np.mean(blk_0202_stacked_extracted_input_features)

0.3602271

In [130]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0202_extracted_input_features.npy", blk_0202_stacked_extracted_input_features)

In [131]:
# extract the targets

In [132]:
%%time
# get the extracted targets

blk_0202_targets = []
for file in blk_0202_output_time_point_images:
    extracted_features = extract_features(file, blk_0202_data_path)
    blk_0202_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.59 s, sys: 132 ms, total: 2.72 s
Wall time: 2.6 s


In [133]:
# stack the extracted features
blk_0202_stacked_extracted_targets = np.stack(blk_0202_targets, axis = 1)

In [134]:
blk_0202_stacked_extracted_targets.shape

(910, 7, 32)

In [135]:
# Store this stack
np.save("seq_2_seq_test_data/block_0202_extracted_target_features.npy", blk_0202_stacked_extracted_targets)

Block 0205

In [136]:
blk_0205_data_path = '../../Spring_2024/S_lab_TasselNet/Block_11_TN/Block_11_images_and_xml'

In [137]:
# we can still use the function above to get the horizontal images
blk_0205_horizontal_images = get_horizontal_images(blk_0205_data_path)

In [138]:
# separate the inputs and the targets
blk_0205_input_time_point_images = blk_0205_horizontal_images[:13]
blk_0205_input_time_point_images.sort()
blk_0205_output_time_point_images = blk_0205_horizontal_images[-7:]
blk_0205_output_time_point_images.sort()

In [139]:
# blk_0205_input_time_point_images, blk_0205_output_time_point_images

In [140]:
%%time
# get the extracted input features

blk_0205_alt_all_features = []
for file in blk_0205_input_time_point_images:
    extracted_features = extract_features(file, blk_0205_data_path)
    blk_0205_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.57 s, sys: 221 ms, total: 4.8 s
Wall time: 4.58 s


In [141]:
# stack the extracted features
blk_0205_stacked_extracted_input_features = np.stack(blk_0205_alt_all_features, axis = 1)

In [142]:
blk_0205_stacked_extracted_input_features.shape

(910, 13, 32)

In [143]:
# check if mean matches
np.mean(blk_0205_stacked_extracted_input_features)

0.37715343

In [144]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0205_extracted_input_features.npy", blk_0205_stacked_extracted_input_features)

In [145]:
# extract the targets

In [146]:
%%time
# get the extracted targets

blk_0205_targets = []
for file in blk_0205_output_time_point_images:
    extracted_features = extract_features(file, blk_0205_data_path)
    blk_0205_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.4 s, sys: 131 ms, total: 2.53 s
Wall time: 2.41 s


In [147]:
# stack the extracted features
blk_0205_stacked_extracted_targets = np.stack(blk_0205_targets, axis = 1)

In [148]:
blk_0205_stacked_extracted_targets.shape

(910, 7, 32)

In [149]:
# Store this stack
np.save("seq_2_seq_test_data/block_0205_extracted_target_features.npy", blk_0205_stacked_extracted_targets)

Block 0206

In [150]:
blk_0206_data_path = '../../Spring_2024/S_lab_TasselNet/Block_12_TN/Block_12_images_and_xml'

In [151]:
# we can still use the function above to get the horizontal images
blk_0206_horizontal_images = get_horizontal_images(blk_0206_data_path)

In [152]:
# separate the inputs and the targets
blk_0206_input_time_point_images = blk_0206_horizontal_images[:13]
blk_0206_input_time_point_images.sort()
blk_0206_output_time_point_images = blk_0206_horizontal_images[-7:]
blk_0206_output_time_point_images.sort()

In [153]:
# blk_0206_input_time_point_images, blk_0206_output_time_point_images

In [154]:
%%time
# get the extracted input features

blk_0206_alt_all_features = []
for file in blk_0206_input_time_point_images:
    extracted_features = extract_features(file, blk_0206_data_path)
    blk_0206_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
CPU times: user 4.62 s, sys: 212 ms, total: 4.83 s
Wall time: 4.61 s


In [155]:
# stack the extracted features
blk_0206_stacked_extracted_input_features = np.stack(blk_0206_alt_all_features, axis = 1)

In [156]:
blk_0206_stacked_extracted_input_features.shape

(910, 13, 32)

In [157]:
# check if mean matches
np.mean(blk_0206_stacked_extracted_input_features)

0.3648797

In [158]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0206_extracted_input_features.npy", blk_0206_stacked_extracted_input_features)

In [159]:
# extract the targets

In [160]:
%%time
# get the extracted targets

blk_0206_targets = []
for file in blk_0206_output_time_point_images:
    extracted_features = extract_features(file, blk_0206_data_path)
    blk_0206_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
CPU times: user 2.53 s, sys: 119 ms, total: 2.65 s
Wall time: 2.53 s


In [161]:
# stack the extracted features
blk_0206_stacked_extracted_targets = np.stack(blk_0206_targets, axis = 1)

In [162]:
blk_0206_stacked_extracted_targets.shape

(910, 7, 32)

In [163]:
# Store this stack
np.save("seq_2_seq_test_data/block_0206_extracted_target_features.npy", blk_0206_stacked_extracted_targets)

Block 0302

In [164]:
blk_0302_data_path = '../../Spring_2024/S_lab_TasselNet/Block_14_TN/Block_14_images_and_xml'

In [165]:
# we can still use the function above to get the horizontal images
blk_0302_horizontal_images = get_horizontal_images(blk_0302_data_path)

In [166]:
# separate the inputs and the targets
blk_0302_input_time_point_images = blk_0302_horizontal_images[:13]
blk_0302_input_time_point_images.sort()
blk_0302_output_time_point_images = blk_0302_horizontal_images[-7:]
blk_0302_output_time_point_images.sort()

In [167]:
# blk_0302_input_time_point_images, blk_0302_output_time_point_images

In [168]:
%%time
# get the extracted input features

blk_0302_alt_all_features = []
for file in blk_0302_input_time_point_images:
    extracted_features = extract_features(file, blk_0302_data_path)
    blk_0302_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.53 s, sys: 250 ms, total: 4.78 s
Wall time: 4.69 s


In [169]:
# stack the extracted features
blk_0302_stacked_extracted_input_features = np.stack(blk_0302_alt_all_features, axis = 1)

In [170]:
blk_0302_stacked_extracted_input_features.shape

(910, 13, 32)

In [171]:
# check if mean matches
np.mean(blk_0302_stacked_extracted_input_features)

0.34874612

In [172]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0302_extracted_input_features.npy", blk_0302_stacked_extracted_input_features)

In [173]:
# extract the targets

In [174]:
%%time
# get the extracted targets

blk_0302_targets = []
for file in blk_0302_output_time_point_images:
    extracted_features = extract_features(file, blk_0302_data_path)
    blk_0302_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.47 s, sys: 144 ms, total: 2.61 s
Wall time: 2.56 s


In [175]:
# stack the extracted features
blk_0302_stacked_extracted_targets = np.stack(blk_0302_targets, axis = 1)

In [176]:
blk_0302_stacked_extracted_targets.shape

(910, 7, 32)

In [177]:
# Store this stack
np.save("seq_2_seq_test_data/block_0302_extracted_target_features.npy", blk_0302_stacked_extracted_targets)

Block 0303

In [178]:
blk_0303_data_path = '../../Spring_2024/S_lab_TasselNet/Block_15_TN/Block_15_images_and_xml'

In [179]:
# we can still use the function above to get the horizontal images
blk_0303_horizontal_images = get_horizontal_images(blk_0303_data_path)

In [180]:
# separate the inputs and the targets
blk_0303_input_time_point_images = blk_0303_horizontal_images[:13]
blk_0303_input_time_point_images.sort()
blk_0303_output_time_point_images = blk_0303_horizontal_images[-7:]
blk_0303_output_time_point_images.sort()

In [181]:
# blk_0303_input_time_point_images, blk_0303_output_time_point_images

In [182]:
%%time
# get the extracted input features

blk_0303_alt_all_features = []
for file in blk_0303_input_time_point_images:
    extracted_features = extract_features(file, blk_0303_data_path)
    blk_0303_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.51 s, sys: 254 ms, total: 4.76 s
Wall time: 4.65 s


In [183]:
# stack the extracted features
blk_0303_stacked_extracted_input_features = np.stack(blk_0303_alt_all_features, axis = 1)

In [184]:
blk_0303_stacked_extracted_input_features.shape

(910, 13, 32)

In [185]:
# check if mean matches
np.mean(blk_0303_stacked_extracted_input_features)

0.35397637

In [186]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0303_extracted_input_features.npy", blk_0303_stacked_extracted_input_features)

In [187]:
# extract the targets

In [188]:
%%time
# get the extracted targets

blk_0303_targets = []
for file in blk_0303_output_time_point_images:
    extracted_features = extract_features(file, blk_0303_data_path)
    blk_0303_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.47 s, sys: 138 ms, total: 2.61 s
Wall time: 2.56 s


In [189]:
# stack the extracted features
blk_0303_stacked_extracted_targets = np.stack(blk_0303_targets, axis = 1)

In [190]:
blk_0303_stacked_extracted_targets.shape

(910, 7, 32)

In [191]:
# Store this stack
np.save("seq_2_seq_test_data/block_0303_extracted_target_features.npy", blk_0303_stacked_extracted_targets)

Block 0304

In [192]:
blk_0304_data_path = '../../Spring_2024/S_lab_TasselNet/Block_16_TN/Block_16_images_and_xml'

In [193]:
# we can still use the function above to get the horizontal images
blk_0304_horizontal_images = get_horizontal_images(blk_0304_data_path)

In [194]:
# separate the inputs and the targets
blk_0304_input_time_point_images = blk_0304_horizontal_images[:13]
blk_0304_input_time_point_images.sort()
blk_0304_output_time_point_images = blk_0304_horizontal_images[-7:]
blk_0304_output_time_point_images.sort()

In [195]:
# blk_0304_input_time_point_images, blk_0304_output_time_point_images

In [196]:
%%time
# get the extracted input features

blk_0304_alt_all_features = []
for file in blk_0304_input_time_point_images:
    extracted_features = extract_features(file, blk_0304_data_path)
    blk_0304_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.63 s, sys: 199 ms, total: 4.83 s
Wall time: 4.73 s


In [197]:
# stack the extracted features
blk_0304_stacked_extracted_input_features = np.stack(blk_0304_alt_all_features, axis = 1)

In [198]:
blk_0304_stacked_extracted_input_features.shape

(910, 13, 32)

In [199]:
# check if mean matches
np.mean(blk_0304_stacked_extracted_input_features)

0.3712321

In [200]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0304_extracted_input_features.npy", blk_0304_stacked_extracted_input_features)

In [201]:
# extract the targets

In [202]:
%%time
# get the extracted targets

blk_0304_targets = []
for file in blk_0304_output_time_point_images:
    extracted_features = extract_features(file, blk_0304_data_path)
    blk_0304_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.59 s, sys: 110 ms, total: 2.7 s
Wall time: 2.66 s


In [203]:
# stack the extracted features
blk_0304_stacked_extracted_targets = np.stack(blk_0304_targets, axis = 1)

In [204]:
blk_0304_stacked_extracted_targets.shape

(910, 7, 32)

In [205]:
# Store this stack
np.save("seq_2_seq_test_data/block_0304_extracted_target_features.npy", blk_0304_stacked_extracted_targets)

Block 0305

In [206]:
blk_0305_data_path = '../../Spring_2024/S_lab_TasselNet/Block_17_TN/Block_17_images_and_xml'

In [207]:
# we can still use the function above to get the horizontal images
blk_0305_horizontal_images = get_horizontal_images(blk_0305_data_path)

In [208]:
# separate the inputs and the targets
blk_0305_input_time_point_images = blk_0305_horizontal_images[:13]
blk_0305_input_time_point_images.sort()
blk_0305_output_time_point_images = blk_0305_horizontal_images[-7:]
blk_0305_output_time_point_images.sort()

In [209]:
# blk_0305_input_time_point_images, blk_0305_output_time_point_images

In [210]:
%%time
# get the extracted input features

blk_0305_alt_all_features = []
for file in blk_0305_input_time_point_images:
    extracted_features = extract_features(file, blk_0305_data_path)
    blk_0305_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.7 s, sys: 207 ms, total: 4.91 s
Wall time: 4.83 s


In [211]:
# stack the extracted features
blk_0305_stacked_extracted_input_features = np.stack(blk_0305_alt_all_features, axis = 1)

In [212]:
blk_0305_stacked_extracted_input_features.shape

(910, 13, 32)

In [213]:
# check if mean matches
np.mean(blk_0305_stacked_extracted_input_features)

0.3665945

In [214]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0305_extracted_input_features.npy", blk_0305_stacked_extracted_input_features)

In [215]:
# extract the targets

In [216]:
%%time
# get the extracted targets

blk_0305_targets = []
for file in blk_0305_output_time_point_images:
    extracted_features = extract_features(file, blk_0305_data_path)
    blk_0305_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.52 s, sys: 127 ms, total: 2.64 s
Wall time: 2.62 s


In [217]:
# stack the extracted features
blk_0305_stacked_extracted_targets = np.stack(blk_0305_targets, axis = 1)

In [218]:
blk_0305_stacked_extracted_targets.shape

(910, 7, 32)

In [219]:
# Store this stack
np.save("seq_2_seq_test_data/block_0305_extracted_target_features.npy", blk_0305_stacked_extracted_targets)

Block 0306

In [220]:
blk_0306_data_path = '../../Spring_2024/S_lab_TasselNet/Block_18_TN/Block_18_images_and_xml'

In [221]:
# we can still use the function above to get the horizontal images
blk_0306_horizontal_images = get_horizontal_images(blk_0306_data_path)

In [222]:
# separate the inputs and the targets
blk_0306_input_time_point_images = blk_0306_horizontal_images[:13]
blk_0306_input_time_point_images.sort()
blk_0306_output_time_point_images = blk_0306_horizontal_images[-7:]
blk_0306_output_time_point_images.sort()

In [223]:
# blk_0306_input_time_point_images, blk_0306_output_time_point_images

In [224]:
%%time
# get the extracted input features

blk_0306_alt_all_features = []
for file in blk_0306_input_time_point_images:
    extracted_features = extract_features(file, blk_0306_data_path)
    blk_0306_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.61 s, sys: 208 ms, total: 4.82 s
Wall time: 4.74 s


In [225]:
# stack the extracted features
blk_0306_stacked_extracted_input_features = np.stack(blk_0306_alt_all_features, axis = 1)

In [226]:
blk_0306_stacked_extracted_input_features.shape

(910, 13, 32)

In [227]:
# check if mean matches
np.mean(blk_0306_stacked_extracted_input_features)

0.36130017

In [228]:
# Store this stack? 
np.save("seq_2_seq_test_data/block_0306_extracted_input_features.npy", blk_0306_stacked_extracted_input_features)

In [229]:
# extract the targets

In [230]:
%%time
# get the extracted targets

blk_0306_targets = []
for file in blk_0306_output_time_point_images:
    extracted_features = extract_features(file, blk_0306_data_path)
    blk_0306_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
CPU times: user 2.53 s, sys: 135 ms, total: 2.66 s
Wall time: 2.62 s


In [231]:
# stack the extracted features
blk_0306_stacked_extracted_targets = np.stack(blk_0306_targets, axis = 1)

In [232]:
blk_0306_stacked_extracted_targets.shape

(910, 7, 32)

In [233]:
# Store this stack
np.save("seq_2_seq_test_data/block_0306_extracted_target_features.npy", blk_0306_stacked_extracted_targets)